In [1]:
import pandas as pd
from pathlib import Path
from pprint import pprint
from datetime import datetime, timedelta # to convert timestamp/ID to human readable format

In [2]:

raw_data = Path('rawdata')
tables = {}
for i, file in enumerate(raw_data.iterdir()):
    name = file.name.split('.')[0]
    tables[f"{name}"] = pd.read_csv(f"rawdata/{file.name}", header = None, names=('ID', name))


In [3]:
columns = [i for i in tables.keys()] 
columns.sort()
pprint(columns)

['fuelDensity',
 'fuelTemp',
 'fuelVolumeFlowRate',
 'inclinometer-raw',
 'latitude',
 'level1median',
 'level2median',
 'longitude',
 'longitudinalWaterSpeed',
 'portPitch',
 'portRudder',
 'speedKmh',
 'speedKnots',
 'starboardPitch',
 'starboardRudder',
 'trackDegreeMagnetic',
 'trackDegreeTrue',
 'trueHeading',
 'windAngle',
 'windSpeed']


### Date transformation functions

In [4]:
def ticks_to_iso(ticks):
    """Convert .NET DateTime.Ticks to an ISO 8601 formatted string."""
    ticks_epoch = datetime(1, 1, 1)  # .NET starts from year 0001-01-01
    date_time = ticks_epoch + timedelta(microseconds=ticks / 10)  # Convert ticks to microseconds
    return date_time.isoformat()

def ticks_to_ymd(ticks):
    """Convert .NET DateTime.Ticks to an ISO 8601 formatted string."""
    ticks_epoch = datetime(1, 1, 1)  # .NET starts from year 0001-01-01
    date_time = ticks_epoch + timedelta(microseconds=ticks / 10)  # Convert ticks to microseconds
    return date_time.strftime("%Y-%m-%d")

# Adds date and ISO date to table in tables dictionary
def add_dates(table_name): 
    tables[table_name]["date"] = tables[table_name].ID.apply(ticks_to_ymd)
    tables[table_name]["iso_date"] = tables[table_name].ID.apply(ticks_to_iso)

### Aggregting date for mass flow of fuel

In [5]:
df_cons = pd.DataFrame(tables["fuelDensity"].ID.tolist(), columns = ('ID',))
df_cons = df_cons.merge(tables["fuelDensity"], on='ID',how='inner')
df_cons = df_cons.merge(tables["fuelVolumeFlowRate"], on='ID',how='inner')
df_cons

,ID,fuelDensity,fuelVolumeFlowRate
0,634019142119225390,0.947109,0.000000
1,634019142129597610,0.947110,0.000000
2,634019142139821660,0.947114,0.000000
3,634019142150036040,0.947104,0.000000
4,634019212935606850,0.938305,0.283816
...,...,...,...
1627319,634067079121428610,0.930151,0.203576
1627320,634067079131642650,0.930103,0.205217
1627321,634067079141866640,0.930075,0.212853
1627322,634067079152080710,0.930082,0.224660


In [6]:
df_cons["date"] = df_cons.ID.apply(ticks_to_ymd)
df_cons["iso_date"] = df_cons.ID.apply(ticks_to_iso)

In [7]:
df_cons

,ID,fuelDensity,fuelVolumeFlowRate,date,iso_date
0,634019142119225390,0.947109,0.000000,2010-02-16,2010-02-16T10:50:11.922536
1,634019142129597610,0.947110,0.000000,2010-02-16,2010-02-16T10:50:12.959760
2,634019142139821660,0.947114,0.000000,2010-02-16,2010-02-16T10:50:13.982168
3,634019142150036040,0.947104,0.000000,2010-02-16,2010-02-16T10:50:15.003600
4,634019212935606850,0.938305,0.283816,2010-02-16,2010-02-16T12:48:13.560688
...,...,...,...,...,...
1627319,634067079121428610,0.930151,0.203576,2010-04-12,2010-04-12T22:25:12.142864
1627320,634067079131642650,0.930103,0.205217,2010-04-12,2010-04-12T22:25:13.164264
1627321,634067079141866640,0.930075,0.212853,2010-04-12,2010-04-12T22:25:14.186664
1627322,634067079152080710,0.930082,0.224660,2010-04-12,2010-04-12T22:25:15.208072


In [8]:
display(df_cons[(df_cons["fuelVolumeFlowRate"] > 0) & (df_cons["date"] == '2010-02-16')])
display(df_cons[(df_cons["fuelVolumeFlowRate"] > 0) &  (df_cons["date"] == '2010-02-17')])

,ID,fuelDensity,fuelVolumeFlowRate,date,iso_date
4,634019212935606850,0.938305,0.283816,2010-02-16,2010-02-16T12:48:13.560688
5,634019212945820680,0.938173,0.279202,2010-02-16,2010-02-16T12:48:14.582064
6,634019212956034910,0.938036,0.282821,2010-02-16,2010-02-16T12:48:15.603488
7,634019212966268860,0.937983,0.280900,2010-02-16,2010-02-16T12:48:16.626888
8,634019212976492840,0.937767,0.279850,2010-02-16,2010-02-16T12:48:17.649280
...,...,...,...,...,...
8634,634019301152989210,0.926645,0.185059,2010-02-16,2010-02-16T15:15:15.298920
8635,634019301163232170,0.926666,0.185255,2010-02-16,2010-02-16T15:15:16.323216
8636,634019301173466570,0.926686,0.187334,2010-02-16,2010-02-16T15:15:17.346656
8637,634019301183680670,0.926721,0.185752,2010-02-16,2010-02-16T15:15:18.368064


,ID,fuelDensity,fuelVolumeFlowRate,date,iso_date
8639,634019858471552070,0.935706,0.394521,2010-02-17,2010-02-17T06:44:07.155208
8640,634019858481776080,0.935774,0.347631,2010-02-17,2010-02-17T06:44:08.177608
8641,634019858492000120,0.935693,0.327961,2010-02-17,2010-02-17T06:44:09.200016
8642,634019858502214030,0.935598,0.317626,2010-02-17,2010-02-17T06:44:10.221400
8643,634019858512448020,0.935773,0.311582,2010-02-17,2010-02-17T06:44:11.244800
...,...,...,...,...,...
36613,634020383895512390,0.925759,0.221838,2010-02-17,2010-02-17T21:19:49.551240
36614,634020383905736690,0.925743,0.221843,2010-02-17,2010-02-17T21:19:50.573672
36615,634020383915950290,0.925741,0.215218,2010-02-17,2010-02-17T21:19:51.595032
36616,634020383926174300,0.925739,0.216107,2010-02-17,2010-02-17T21:19:52.617432


#### Adding data about motion
Since fuel density is implied by fuel temperature. These variables are correlated and adding tempereture would not add any information. I will rather add location as it can be more related to the weather conditions, that can have influence on fuel mass flow. Since joining on time stamp is not an option in this case, I will transform it to human readable format and 

In [9]:
add_dates("longitude")
add_dates("latitude")

In [10]:
df_longitude = tables["longitude"]
df_longitude["iso_date_min"] = df_longitude.iso_date.apply(lambda x: x[:16])
df_longitude

,ID,longitude,date,iso_date,iso_date_min
0,634018095137704470,00649.1143W,2010-02-15,2010-02-15T05:45:13.770448,2010-02-15T05:45
1,634018095137898920,00649.1143W,2010-02-15,2010-02-15T05:45:13.789888,2010-02-15T05:45
2,634018095146534160,00649.1143W,2010-02-15,2010-02-15T05:45:14.653416,2010-02-15T05:45
3,634018095157587450,00649.1144W,2010-02-15,2010-02-15T05:45:15.758744,2010-02-15T05:45
4,634018095158588070,00649.1144W,2010-02-15,2010-02-15T05:45:15.858808,2010-02-15T05:45
...,...,...,...,...,...
2677691,634067079142236760,00649.1087W,2010-04-12,2010-04-12T22:25:14.223680,2010-04-12T22:25
2677692,634067079143136790,00649.1087W,2010-04-12,2010-04-12T22:25:14.313680,2010-04-12T22:25
2677693,634067079152991490,00649.1088W,2010-04-12,2010-04-12T22:25:15.299152,2010-04-12T22:25
2677694,634067079154070470,00649.1088W,2010-04-12,2010-04-12T22:25:15.407048,2010-04-12T22:25


In [11]:
print("First fuel consumption timestemp 2010-02-16T12:48:13.560688")
print("Longitude")
display(df_longitude[df_longitude.iso_date_min == '2010-02-16T12:48'])
print("Latitude")
display(tables["latitude"])

First fuel consumption timestemp 2010-02-16T12:48:13.560688
Longitude


,ID,longitude,date,iso_date,iso_date_min
81290,634019212953686680,00645.9397W,2010-02-16,2010-02-16T12:48:15.368672,2010-02-16T12:48
81291,634019212954187230,00645.9397W,2010-02-16,2010-02-16T12:48:15.418720,2010-02-16T12:48
81292,634019212954766060,00645.9397W,2010-02-16,2010-02-16T12:48:15.476608,2010-02-16T12:48
81293,634019212972795710,00645.9408W,2010-02-16,2010-02-16T12:48:17.279568,2010-02-16T12:48
81294,634019212973845470,00645.9408W,2010-02-16,2010-02-16T12:48:17.384544,2010-02-16T12:48
...,...,...,...,...,...
81356,634019213372280600,00645.9626W,2010-02-16,2010-02-16T12:48:57.228064,2010-02-16T12:48
81357,634019213373190630,00645.9626W,2010-02-16,2010-02-16T12:48:57.319064,2010-02-16T12:48
81358,634019213384673970,00645.9632W,2010-02-16,2010-02-16T12:48:58.467400,2010-02-16T12:48
81359,634019213392539370,00645.9636W,2010-02-16,2010-02-16T12:48:59.253936,2010-02-16T12:48


Latitude


,ID,latitude,date,iso_date
0,634018095137704470,6132.9236N,2010-02-15,2010-02-15T05:45:13.770448
1,634018095137898920,6132.9236N,2010-02-15,2010-02-15T05:45:13.789888
2,634018095146534160,6132.9240N,2010-02-15,2010-02-15T05:45:14.653416
3,634018095157587450,6132.9243N,2010-02-15,2010-02-15T05:45:15.758744
4,634018095158588070,6132.9243N,2010-02-15,2010-02-15T05:45:15.858808
...,...,...,...,...
2677708,634067079142236760,6132.9260N,2010-04-12,2010-04-12T22:25:14.223680
2677709,634067079143136790,6132.9260N,2010-04-12,2010-04-12T22:25:14.313680
2677710,634067079152991490,6132.9255N,2010-04-12,2010-04-12T22:25:15.299152
2677711,634067079154070470,6132.9255N,2010-04-12,2010-04-12T22:25:15.407048


In [12]:
# I cannot joinon date ofc
'''
df_gps = pd.DataFrame(tables["latitude"].date.tolist(), columns = ('date',))
df_gps = df_gps.merge(tables["latitude"].drop("ID", axis=1), on='date',how='inner')
df_gps = df_gps.merge(tables["longitude"].drop("ID", axis=1), on='date',how='inner')
df_gps
'''

'\ndf_gps = pd.DataFrame(tables["latitude"].date.tolist(), columns = (\'date\',))\ndf_gps = df_gps.merge(tables["latitude"].drop("ID", axis=1), on=\'date\',how=\'inner\')\ndf_gps = df_gps.merge(tables["longitude"].drop("ID", axis=1), on=\'date\',how=\'inner\')\ndf_gps\n'

In [13]:
display(tables["latitude"].drop("ID", axis=1))

,latitude,date,iso_date
0,6132.9236N,2010-02-15,2010-02-15T05:45:13.770448
1,6132.9236N,2010-02-15,2010-02-15T05:45:13.789888
2,6132.9240N,2010-02-15,2010-02-15T05:45:14.653416
3,6132.9243N,2010-02-15,2010-02-15T05:45:15.758744
4,6132.9243N,2010-02-15,2010-02-15T05:45:15.858808
...,...,...,...
2677708,6132.9260N,2010-04-12,2010-04-12T22:25:14.223680
2677709,6132.9260N,2010-04-12,2010-04-12T22:25:14.313680
2677710,6132.9255N,2010-04-12,2010-04-12T22:25:15.299152
2677711,6132.9255N,2010-04-12,2010-04-12T22:25:15.407048


In [14]:
tables["trueHeading"].ID.apply

<bound method Series.apply of 0          634018095140997220
1          634018095152989600
2          634018095164992880
3          634018095176985300
4          634018095188988420
                  ...        
1439534    634067079106147630
1439535    634067079118151620
1439536    634067079130153750
1439537    634067079142146670
1439538    634067079154149980
Name: ID, Length: 1439539, dtype: int64>

## Count records per day

In [17]:
for name, df in tables.items():
    print(name)
    display(df.head(5))

starboardRudder


,ID,starboardRudder
0,634018095135582500,3.142
1,634018095145588030,3.200
2,634018095155594280,3.202
3,634018095165605510,3.076
4,634018095175611210,3.078


portPitch


,ID,portPitch
0,634018095135582500,2.127
1,634018095145588030,2.156
2,634018095155594280,2.295
3,634018095165605510,2.446
4,634018095175611210,2.469


level2median


...

,ID,windAngle
0,634018095148473090,322
1,634018095168491190,330
2,634018095188489250,325
3,634018095208487470,328
4,634018095228465720,350


speedKmh


,ID,speedKmh
0,634018095130693130,1.1
1,634018095139168740,1.1
2,634018095149771410,1.3
3,634018095160296020,1.5
4,634018095160835260,1.5


fuelTemp


,ID,fuelTemp
0,634019142119225390,62.20651
1,634019142129597610,62.20406
2,634019142139821660,62.19989
3,634019142150036040,62.20412
4,634019212935606850,86.89216


In [20]:
df_rec_counts = None
for name, df in tables.items():
    print(f'Processing {name}')
    if "date" not in df.keys():
        add_dates(name)
    print('Counting...')
    df = df[["date", name]].groupby("date").count()
    
    if df_rec_counts is None:
        df_rec_counts = df
    else:
        df_rec_counts = pd.merge(df_rec_counts, df, on="date")

display(df_rec_counts)
df_rec_counts.to_csv('records_per_day.csv', header=True)

Processing starboardRudder
Counting...
Processing portPitch
Counting...
Processing level2median
Counting...
Processing fuelDensity
Counting...
Processing latitude
Counting...
Processing speedKnots
Counting...
Processing longitudinalWaterSpeed
Counting...
Processing trackDegreeTrue
Counting...
Processing windSpeed
Counting...
Processing trueHeading
Counting...
Processing starboardPitch
Counting...
Processing portRudder
Counting...
Processing longitude
Counting...
Processing fuelVolumeFlowRate
Counting...
Processing level1median
Counting...
Processing trackDegreeMagnetic
Counting...
Processing inclinometer-raw
Counting...
Processing windAngle
Counting...
Processing speedKmh
Counting...
Processing fuelTemp
Counting...


,starboardRudder,portPitch,level2median,fuelDensity,latitude,speedKnots,longitudinalWaterSpeed,trackDegreeTrue,windSpeed,trueHeading,starboardPitch,portRudder,longitude,fuelVolumeFlowRate,level1median,trackDegreeMagnetic,inclinometer-raw,windAngle,speedKmh,fuelTemp
date,,,,,,,,,,,,,,,,,,,,
2010-02-16,18557,18557,37535,8639,28748,18972,5894,18969,9283,15468,18557,18557,28748,8639,37902,18969,38242,9283,18972,8639
2010-02-17,28579,28579,60166,27979,44339,29247,9083,29240,14299,23833,28579,28579,44337,27979,63204,29240,58934,14299,29247,27979
2010-02-18,25999,25999,55414,25270,40377,26627,8255,26620,13009,21673,25999,25999,40375,25270,60086,26619,53587,13009,26627,25270
2010-02-19,35029,35029,70955,34294,54387,35885,11130,35873,17530,29216,35029,35029,54388,34294,73426,35874,72241,17530,35886,34294
2010-02-20,35461,35461,72709,34724,55045,36347,11271,36341,17753,29584,35461,35461,55045,34724,76399,36341,73143,17753,36348,34724
2010-02-21,29608,29608,48878,28988,45936,30302,9407,30296,14818,24696,29608,29608,45936,28988,45516,30296,61059,14818,30302,28988
2010-02-22,39748,39748,81014,34509,61703,40666,12629,40656,19892,33148,39748,39748,61704,34509,88646,40657,81964,19892,40665,34509
2010-02-23,26567,26567,58848,19360,41184,27197,8440,27190,13294,22152,26567,26567,41184,19360,63085,27190,54774,13294,27197,19360
2010-02-24,29573,29573,61634,28953,45826,30220,9395,30211,14797,24662,29573,29573,45827,28953,66193,30211,60985,14797,30220,28953


In [ ]:
print(df_rec_counts.shape)
display(df_rec_counts.info())
display(df_rec_counts.describe())

### Finds days with incomplete records


In [21]:
def_rec_Null_counts = None
for name, df in tables.items():
    print(f'Processing {name}')
    if "date" not in df.keys():
        add_dates(name)
    print('Counting...')
    df = df[["date", name]].groupby("date").count()
    
    if def_rec_Null_counts is None:
        def_rec_Null_counts = df
    else:
        def_rec_Null_counts = pd.merge(def_rec_Null_counts, df, on="date", how="outer")

display(def_rec_Null_counts)

Processing starboardRudder
Counting...
Processing portPitch
Counting...
Processing level2median
Counting...
Processing fuelDensity
Counting...
Processing latitude
Counting...
Processing speedKnots
Counting...
Processing longitudinalWaterSpeed
Counting...
Processing trackDegreeTrue
Counting...
Processing windSpeed
Counting...
Processing trueHeading
Counting...
Processing starboardPitch
Counting...
Processing portRudder
Counting...
Processing longitude
Counting...
Processing fuelVolumeFlowRate
Counting...
Processing level1median
Counting...
Processing trackDegreeMagnetic
Counting...
Processing inclinometer-raw
Counting...
Processing windAngle
Counting...
Processing speedKmh
Counting...
Processing fuelTemp
Counting...


,starboardRudder,portPitch,level2median,fuelDensity,latitude,speedKnots,longitudinalWaterSpeed,trackDegreeTrue,windSpeed,trueHeading,starboardPitch,portRudder,longitude,fuelVolumeFlowRate,level1median,trackDegreeMagnetic,inclinometer-raw,windAngle,speedKmh,fuelTemp
date,,,,,,,,,,,,,,,,,,,,
2010-02-15,42697,42697,96000,NaN,66202,43711,13453,43700,21365,35604,42697,42697,66202,NaN,94946,43700,88044,21365,43710,NaN
2010-02-16,18557,18557,37535,8639.0,28748,18972,5894,18969,9283,15468,18557,18557,28748,8639.0,37902,18969,38242,9283,18972,8639.0
2010-02-17,28579,28579,60166,27979.0,44339,29247,9083,29240,14299,23833,28579,28579,44337,27979.0,63204,29240,58934,14299,29247,27979.0
2010-02-18,25999,25999,55414,25270.0,40377,26627,8255,26620,13009,21673,25999,25999,40375,25270.0,60086,26619,53587,13009,26627,25270.0
2010-02-19,35029,35029,70955,34294.0,54387,35885,11130,35873,17530,29216,35029,35029,54388,34294.0,73426,35874,72241,17530,35886,34294.0
2010-02-20,35461,35461,72709,34724.0,55045,36347,11271,36341,17753,29584,35461,35461,55045,34724.0,76399,36341,73143,17753,36348,34724.0
2010-02-21,29608,29608,48878,28988.0,45936,30302,9407,30296,14818,24696,29608,29608,45936,28988.0,45516,30296,61059,14818,30302,28988.0
2010-02-22,39748,39748,81014,34509.0,61703,40666,12629,40656,19892,33148,39748,39748,61704,34509.0,88646,40657,81964,19892,40665,34509.0
2010-02-23,26567,26567,58848,19360.0,41184,27197,8440,27190,13294,22152,26567,26567,41184,19360.0,63085,27190,54774,13294,27197,19360.0
